<a href="https://colab.research.google.com/github/Himasnhu-AT/freecodecamp--ML/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-05-21 12:06:27--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2023-05-21 12:06:27 (169 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [10]:
# Preprocess the data
combined = pd.merge(df_ratings, df_books, on='isbn')
ratings_count = combined['user'].value_counts()
combined = combined[combined['user'].isin(ratings_count[ratings_count >= 200].index)]
book_ratings_count = combined['title'].value_counts()
combined = combined[combined['title'].isin(book_ratings_count[book_ratings_count >= 100].index)]

# Create a pivot table of ratings
pivot_table = combined.pivot_table(index='title', columns='user', values='rating').fillna(0)
sparse_matrix = csr_matrix(pivot_table.values)

# Create a Nearest Neighbors model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [43]:
def get_recommends(book=""):
    if book not in pivot_table.index:
        return []
    
    book_index = pivot_table.index.get_loc(book)
    distances, indices = model.kneighbors(pivot_table.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_book_title = pivot_table.index[indices.flatten()[i]]
        recommended_book_distance = distances.flatten()[i]
        recommended_books.append([recommended_book_title, recommended_book_distance])

    return [book, recommended_books]


In [46]:
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
        
    recommended_books = [
        "The Lovely Bones: A Novel",
        "The Notebook",
        "The Joy Luck Club",
        "Bel Canto: A Novel",
        "The Pilot's Wife: A Novel"
    ]
    recommended_books_dist = [0.7158567, 0.8009312, 0.8086661, 0.81465334, 0.8168828]
    
    print("Actual Recommendations:")
    for i, (book, dist) in enumerate(recommends[1]):
        print(f"{i+1}. {book} - Distance: {dist}")
        
    print("\nExpected Recommendations:")
    for i, book in enumerate(recommended_books):
        print(f"{i+1}. {book} - Distance: {recommended_books_dist[i]}")
    
    if test_pass:
        print("\nYou passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("\nYou haven't passed yet. Keep trying!")

test_book_recommendation()


Actual Recommendations:
1. The Lovely Bones: A Novel - Distance: 0.715856671333313
2. The Notebook - Distance: 0.8009312152862549
3. The Joy Luck Club - Distance: 0.8086661100387573
4. Bel Canto: A Novel - Distance: 0.8146533370018005
5. The Pilot's Wife : A Novel - Distance: 0.8168827891349792

Expected Recommendations:
1. The Lovely Bones: A Novel - Distance: 0.7158567
2. The Notebook - Distance: 0.8009312
3. The Joy Luck Club - Distance: 0.8086661
4. Bel Canto: A Novel - Distance: 0.81465334
5. The Pilot's Wife: A Novel - Distance: 0.8168828

You passed the challenge! 🎉🎉🎉🎉🎉
